<h1 style="text-align: center; margin-bottom: 10px;">Improving the Bounds of Spencer's Graph</h1>
<h3 style="text-align: center; margin-top: 10px;">Name: Charles Lin</h3>

Hello World!

<h2 style="text-align: center;">Abstract</h2>
The field of Ramsey Theory has allowed us to find various graphs with unique properties. However, many bounds for even small Ramsey values (such as R(5)) are still undetermined and remain as open questions in the world of mathematics. This project explores finding a graph that doesn't have a $K_4$ clique AND for any 2-coloring of its edges, there exists a monochromatic triangle within the graph. A bound for the number of vertices for a graph that holds this property has already been proven ($3 \times 10^8$), but we aim to significantly improve that bound using randomized algorithm techniques in this project.

<h2 style="text-align: center;">Introduction Background</h2>
Before we dive into how we will use randomized algorithms to solve this problem, keep in mind the following definitions/theorems related to this problem:
<h3 style="text-align: center;">Definitions and Theorems</h3>

* The **Random Graph** $G(n,p)$ is an *undirected* graph on n vertices (i.e. $V=[n]$) and whose edge set $E$ contains an edge $(x,y)\in \binom{V}{2}$ with probability p.
* $\text{RAM}(G)$ indicates that for all 2 colorings $\text{COL}\colon \binom{V}{2}\rightarrow [2]$ for the edges of a graph G, there exists a monochromatic triangle.
* The set $U$ is defined as:
$$U = \bigcup _{x,y,z\in \binom{V}{3}}\{(x,xyz)\colon xyz \text{ is a Δ in G}\}$$
* And for $x\in V$, the functions $U(x), N(x)$ are defined as:
$$U(x) = \{(x,xyz)\colon xyz \text{ is a Δ in G}\}$$
$$N(x) = \{y\colon (x,y) \in E\}$$
* $U^{COL}$ is a subset of $U$ that uses an edge coloring of the graph $\text{COL}\colon \binom{V}{2} \rightarrow [2]$defined as:
$$U^{COL} = \bigcup _{x,y,z\in \binom{V}{3}}\{(x,xyz)\colon xyz \text{ is a Δ in G } \& \text{ COL}(x,y) \not = \text{COL}(x,z)\}$$

* With these definitions comes the following theorems (proofs ommitted):
  * **Thm:** $|U^{COL}|=\frac{2}{3}|U|$
  * **Thm:** Let $x \in V$, $R(x)=\{y\colon \text{COL}(x,y)=R\}$, and $B(x)=\{z\colon \text{COL}(x,z)=B\}$. It follows that:

1) $|U^{COL}|=|\{(y,z)\in E\colon y\in R(x) \land z \in B(x)\}|$
2) $|U^{COL}|\leq \max_{N(x)=Y\cup Z}|\{(y,z)\in E\colon y\in Y \land z \in Z\}|$

* We can now state our final definition $A(x)$:
$$A(x)=\max_{N(x)=R\cup B}\{(y,z)\in E\colon y\in R \land z \in B\}$$

* And with some mathemagic and algebraic manipulation, this defintion provides our star theorem for the overall purpose of this project:
  * **Thm:** If $\sum_{x\in V}A(x) < \frac{2}{3}|U|$, then $\text{RAM}(G)$.
 
---
With our golden theorem, we now have a way to determine our criteria, $\text{RAM}(G)$, without having to check every 2-coloring E for a given graph G. The following sections use randomized algorithms for graph creation to attempt to find a better graph that achieves this criteria.

<h2 style="text-align: center;">The Program</h2>
The following program below is build in a randomized way to attempt to find a suitable graph G with a better n bound. It is built in the following way:

**Inputs:** $n\in \mathbb{N}$, $0 < p < 1$, and $I$. (Example: $n = 100$, $p = 1/10$, $I = 1000$)

**Pseudocode Process:**
While a suitable G hasn't been found or the below process hasn't been attempted I times:

1) Create a **Random Graph** $G'=G(n, p)$
2) Check to see if there are $K_4$'s in $G'$. If there are, randomly delete an edge from a $K_4$ until there are no more $K_4$'s
3) Call this new graph G, and compute $\sum_{x\in V}A(x)$ and $\frac{2}{3}|U|$. If our golden criteria is reached ($\sum_{x\in V}A(x)$ and < $\frac{2}{3}|U|$), HOORAY!

Now we will finally get down and dirty with some code:

In [1]:
import random
import time
import math
from itertools import combinations, product
import networkx as nx
import numpy as np
from typing import Tuple
import cvxpy as cvx
from IPython.display import display
import pandas as pd

In [2]:
# Creates a Random Graph (adjacency matrix) as detailed in definitions
def randomGraph(n, p):
    x = [[0]*n for _ in range(n)]
    for i in range(n - 1):
        for j in range(i + 1, n):
            if random.random() < p:
                x[i][j] = 1
                x[j][i] = 1
    return x

In [3]:
# Builds a list of all possible 3-subsets up to n
def subsets3(n):
    x = []
    for i in range(n - 2):
        for j in range(i + 1, n - 1):
            for k in range(j + 1, n):
                x.append((i,j,k))
    return x

In [4]:
# Builds a list of all possible 4-subsets up to n
def subsets4(n):
    x = []
    for i in range(n - 3):
        for j in range(i + 1, n - 2):
            for k in range(j + 1, n - 1):
                for l in range (k + 1, n):
                    x.append((i,j,k,l))
    return x

In [5]:
# Create a list of all the K_4's present in a graph G
def cliques4(G, n):
    t1 = time.time()
    subsets = subsets4(n)
    t2 = time.time()
    cliques = []
    for ss in subsets:
        check = G[ss[0]][ss[1]] + G[ss[0]][ss[2]] + G[ss[0]][ss[3]] + G[ss[1]][ss[2]] + G[ss[1]][ss[3]] + G[ss[2]][ss[3]]
        if check == 6:
            cliques.append(ss)
    t3 = time.time()
    print(f"sub4sets takes {t2 - t1} ; finding cliques takes {t3 - t2}")
    return cliques

In [6]:
# Create a graph G with no 4-cliques:
def no4CliqueG(n, p):
    G = randomGraph(n, p)
    cliques = cliques4(G, n)
    for cq in cliques:
        # Check to see if a previous removal already eliminated this specific clique
        if G[cq[0]][cq[1]] + G[cq[0]][cq[2]] + G[cq[0]][cq[3]] + G[cq[1]][cq[2]] + G[cq[1]][cq[3]] + G[cq[2]][cq[3]] != 6:
            continue
        else:
            # Otherwise, randomly remove an edge from the 4-clique to eliminate it
            match int(random.random()*6):
                case 0:
                    G[cq[0]][cq[1]] = 0
                    G[cq[1]][cq[0]] = 0
                case 1:
                    G[cq[0]][cq[2]] = 0
                    G[cq[2]][cq[0]] = 0
                case 2:
                    G[cq[0]][cq[3]] = 0
                    G[cq[3]][cq[0]] = 0
                case 3:
                    G[cq[1]][cq[2]] = 0
                    G[cq[2]][cq[1]] = 0
                case 4:
                    G[cq[1]][cq[3]] = 0
                    G[cq[3]][cq[1]] = 0
                case _:
                    G[cq[2]][cq[3]] = 0
                    G[cq[3]][cq[2]] = 0
    return G

In [7]:
# Determine all 2-partitions of a given set (needed for ∑A(x))
def twoParts(s):
    s = list(s)
    seen = set()
    n = len(s)
    
    # Generate all possible non-empty subsets for the first part (up to half the size to avoid duplicates)
    for i in range(1, n // 2 + 1):
        for subset in combinations(s, i):
            part1 = set(subset)
            part2 = set(s) - part1
            # Sort to normalize and avoid mirror duplicates
            key = frozenset([frozenset(part1), frozenset(part2)])
            if key not in seen:
                seen.add(key)
                yield (part1, part2)

In [8]:
# Calculates ∑A(x) for a 4-cliqueless graph G
def sumAx(G, n):
    # Seeing every vertex x in V
    count = 0
    for i in range(n):
        # If the x has no edges to other verticies, it's irrelevant
        if max(G[i]) == 0:
            continue
        # Find the neighbors of x
        neighbors = set()
        for j in range(n):
            if G[i][j] == 1:
                neighbors.add(j)
        # For all 2-partitions of the neighbor set, determine the amount of edges present and get the max:
        maX = 0
        for p1, p2 in twoParts(neighbors):
            curr = 0
            # For a given partition, find how many edges exist with one vertex in p1 and the other in p2
            for v1 in p1:
                for v2 in p2:
                    if G[v1][v2] == 1:
                        curr += 1
            # Update maximum
            maX = curr if curr > maX else maX
        # Sum A(x) over all x in V
        count += maX
    return count

In [9]:
# Calculates ∑A(x) for a 4-cliqueless graph G
def sumAx_rand(G, n):
    # Seeing every vertex x in V
    count = 0
    for i in range(n):
        # If the x has no edges to other verticies, it's irrelevant
        if max(G[i]) == 0:
            continue
        # Find the neighbors of x
        neighbors = set()
        for j in range(n):
            if G[i][j] == 1:
                neighbors.add(j)

        S, T = set(), set()
        for v in neighbors:
            if random.random() > 0.5:
                S.add(v)
            else:
                T.add(v)
    
        # Step 2: Count the edges crossing the partition
        cut_size = 0
        for u in S:
            for v in T:
                if G[u][v] == 1:
                    cut_size += 1

        # Sum A(x) over all x in V
        count += cut_size
    return count

In [10]:
G = no4CliqueG(100, 0.2)
time1 = time.time()
uno = sumAx(G, 50)
time2 = time.time()
print(f"Naive sumAx: {time2 - time1} secs; {uno}")
time3 = time.time()
dos = sumAx_rand(G, 50)
time4 = time.time()
print(f"Faster sumAx: {time4 - time3} secs; {dos}")

sub4sets takes 0.8534562587738037 ; finding cliques takes 1.085005760192871
Naive sumAx: 0.3565521240234375 secs; 255
Faster sumAx: 0.0007390975952148438 secs; 126


In [11]:
# Note that |U| = |∆'s in G * 3|. This function returns 2/3|U|
def size2_3U(G, n):
    subsets = subsets3(n)
    count = 0
    for subset in subsets:
        check = G[subset[0]][subset[1]] + G[subset[0]][subset[2]] + G[subset[1]][subset[2]]
        if check == 3:
            count += 1
    return count * 2

We now have all our necessary functions required to use randomized algorithms to possibly find a viable graph. Let us implement the overall algorithm for multiple iterations I:

In [12]:
# Overall Program detailed in above Pseudocode
def PROG(n, p, I):
    closest = (1000000, 0)
    # bestG = None
    for _ in range(I):
        time1 = time.time()
        G = no4CliqueG(n, p)
        time2 = time.time()
        print(f"No 4-clique Graph Creation takes {time2 - time1} secs")
        Asum = sumAx(G, n)
        time3 = time.time()
        print(f"∑A(x) takes {time3 - time2} secs")
        U2_3 = size2_3U(G, n)
        time4 = time.time()
        print(f"2/3|U| takes {time4 - time3} secs")
        if (Asum - U2_3) < (closest[0] - closest[1]):
            closest = (Asum, U2_3)
            # bestG = G
    return closest

In [13]:
# Overall Program detailed in above Pseudocode
def PROG_rand(n, p, I):
    closest = (1000000, 0)
    # bestG = None
    for _ in range(I):
        G = no4CliqueG(n, p)
        # The times two is indicative of the randomized algorithm's expected value being 1/2 the actual max cut value
        Asum = sumAx_rand(G, n) * 2
        U2_3 = size2_3U(G, n)
        if (Asum - U2_3) < (closest[0] - closest[1]):
            closest = (Asum, U2_3)
            # bestG = G
    return closest

In [14]:
# Example Usage with a small amount of vertices
PROG(50, 0.4, 1)

sub4sets takes 0.058548927307128906 ; finding cliques takes 0.05160999298095703
No 4-clique Graph Creation takes 0.1183319091796875 secs
∑A(x) takes 6.018474340438843 secs
2/3|U| takes 0.09231185913085938 secs


(858, 620)

In [15]:
PROG_rand(50, 0.4, 1)

sub4sets takes 0.06722784042358398 ; finding cliques takes 0.07615423202514648


(818, 556)

<h2 style="text-align: center;">Trying to find a better n</h2>
Now that we have PROG, I will use careful analysis and techniques to narrow down ways to find a suitable G and better n bound. First, lets try and tinker with the value of p for certain values of n to get the best p to move forward with:

In [ ]:
# n = 50, I = 25, p in {0.1,0.2,...,0.9}
uno = "p"
dos = "∑A(x)"
tres = "2/3|U|"
quatro = "∑A(x) - 2/3|U|"
cinqo = "Closeness"
data = {uno:[],
        dos:[],
        tres:[],
        quatro:[],
        cinqo:[]}
for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    closest = PROG_rand(100, p, 10)
    data[uno].append(f"{p:.1f}")
    data[dos].append(closest[0])
    data[tres].append(closest[1])
    data[quatro].append(closest[0] - closest[1])
    data[cinqo].append(f"{(closest[0] - closest[1])/(closest[0] + closest[1]):.2f}")

df = pd.DataFrame(data)
display(df.style.hide(axis="index"))

sub4sets takes 0.818709135055542 ; finding cliques takes 0.8908460140228271
sub4sets takes 0.649979829788208 ; finding cliques takes 0.9105992317199707
sub4sets takes 0.5201117992401123 ; finding cliques takes 0.7687931060791016
sub4sets takes 0.5430238246917725 ; finding cliques takes 0.7708151340484619
sub4sets takes 0.6215739250183105 ; finding cliques takes 0.8808350563049316
sub4sets takes 0.6675310134887695 ; finding cliques takes 0.8359301090240479
sub4sets takes 0.5268611907958984 ; finding cliques takes 0.8416767120361328
sub4sets takes 0.5396013259887695 ; finding cliques takes 0.7665636539459229
sub4sets takes 0.5368530750274658 ; finding cliques takes 0.8717391490936279
sub4sets takes 0.5940990447998047 ; finding cliques takes 0.9351367950439453
sub4sets takes 0.6751229763031006 ; finding cliques takes 1.0497510433197021
sub4sets takes 0.574955940246582 ; finding cliques takes 0.7936301231384277
sub4sets takes 0.570573091506958 ; finding cliques takes 0.8597347736358643
sub

So double sided news: it appears that the value of p doesn't seem to have much of a bearing on the closesness of ∑A(x) and 2/3|U|; HOWEVER, that is also good news because it means smaller values of p (which lead to less computation time) are just as probably good. We can continue with a new G_PROG that possibly gives us a valid graph :)

In [ ]:
def G_PROG(n, p, I):
    closest = (1000000, 0)
    viableG = None
    for _ in range(I):
        t1 = time.time()
        G = no4CliqueG(n, p)
        # The times two is indicative of the randomized algorithm's expected value being 1/2 the actual max cut value
        t2 = time.time()
        print(f"noCliques takes {t2-t1} seconds")
        Asum = sumAx_rand(G, n) * 2
        t3 = time.time()
        print(f"noCliques takes {t3-t2} seconds")
        U2_3 = size2_3U(G, n)
        t4 = time.time()
        print(f"U2_3 takes {t4-t3} seconds")
        if (Asum - U2_3) < (closest[0] - closest[1]):
            closest = (Asum, U2_3)
        if (Asum - U2_3) < 0:
            viableG = G
    
    return closest, viableG

In [ ]:
# Run G_PROG for a variety of N and I:
oogaBooga = None
uno = "n"
dos = "∑A(x)"
tres = "2/3|U|"
quatro = "∑A(x) - 2/3|U|"
cinqo = "Closeness"
data = {uno:[],
        dos:[],
        tres:[],
        quatro:[],
        cinqo:[]}
for n in range(100, 200, 50):
    closest, viableG = G_PROG(n, 0.2, 1)
    if not viableG == None:
        oogaBooga = viableG
    data[uno].append(n)
    data[dos].append(closest[0])
    data[tres].append(closest[1])
    data[quatro].append(closest[0] - closest[1])
    data[cinqo].append(f"{(closest[0] - closest[1])/(closest[0] + closest[1]):.2f}")

df = pd.DataFrame(data)
display(df.style.hide(axis="index"))

<h2 style="text-align: center;">Discussion</h2>
One of the most challenging issues related to finidng a good bound for n is obviously the computational limits of python. A few key optimization processes I used are detailed below.

When determining what parts of the code prove to be a bottleneck, it is helpful to see what segments in PROG are the most taxing using time statements:

In [ ]:
# Overall Program detailed in above Pseudocode
def PROG(n, p, I):
    closest = (1000000, 0)
    # bestG = None
    for _ in range(I):
        time1 = time.time()
        G = no4CliqueG(n, p)
        time2 = time.time()
        print(f"No 4-clique Graph Creation takes {time2 - time1} secs")
        Asum = sumAx(G, n)
        time3 = time.time()
        print(f"∑A(x) takes {time3 - time2} secs")
        U2_3 = size2_3U(G, n)
        time4 = time.time()
        print(f"2/3|U| takes {time4 - time3} secs")
        if (Asum - U2_3) < (closest[0] - closest[1]):
            closest = (Asum, U2_3)
            # bestG = G
    return closest

One should note that determining the value of $A(x)=\max_{N(x)=R\cup B}\{(y,z)\in E\colon y\in R \land z \in B\}$ is essentially the same as solving the max-cut problem... which is NP hard :,). HOWEVER, there is a very quick polynomial time randomized algorithm that can approximate the max-cut of a given graph with expected value 1/2 of the ACTUAL max-cut. THUS, I nested two randomized algorithms to significantly reduce the runtime of the overall algorithm!

In [ ]:
# Time to optimize because we clearly cannot continue until we can confidently reach significantly larger bounds on this stuff.

# Run G_PROG for a variety of N and I:
oogaBooga = None
uno = "n"
dos = "∑A(x)"
tres = "2/3|U|"
quatro = "∑A(x) - 2/3|U|"
cinqo = "Closeness"
data = {uno:[],
        dos:[],
        tres:[],
        quatro:[],
        cinqo:[]}
for n in range(100, 200, 50):
    closest, viableG = G_PROG(n, 0.2, 1)
    if not viableG == None:
        oogaBooga = viableG
    data[uno].append(n)
    data[dos].append(closest[0])
    data[tres].append(closest[1])
    data[quatro].append(closest[0] - closest[1])
    data[cinqo].append(f"{(closest[0] - closest[1])/(closest[0] + closest[1]):.2f}")

In [ ]:
# GRAPH CREATION IS THE BANE OF ME NOW TO IMPROVE

In [ ]:
# Creates a Random Graph (adjacency matrix) as detailed in definitions
# Create a graph G with no 4-cliques:
def no4CliqueG_analyze(n, p):
    t1 = time.time()
    G = randomGraph(n, p)
    t2 = time.time()
    cliques = cliques4(G, n)
    t3 = time.time()
    for cq in cliques:
        # Check to see if a previous removal already eliminated this specific clique
        if G[cq[0]][cq[1]] + G[cq[0]][cq[2]] + G[cq[0]][cq[3]] + G[cq[1]][cq[2]] + G[cq[1]][cq[3]] + G[cq[2]][cq[3]] != 6:
            continue
        else:
            # Otherwise, randomly remove an edge from the 4-clique to eliminate it
            match int(random.random()*6):
                case 0:
                    G[cq[0]][cq[1]] = 0
                    G[cq[1]][cq[0]] = 0
                case 1:
                    G[cq[0]][cq[2]] = 0
                    G[cq[2]][cq[0]] = 0
                case 2:
                    G[cq[0]][cq[3]] = 0
                    G[cq[3]][cq[0]] = 0
                case 3:
                    G[cq[1]][cq[2]] = 0
                    G[cq[2]][cq[1]] = 0
                case 4:
                    G[cq[1]][cq[3]] = 0
                    G[cq[3]][cq[1]] = 0
                case _:
                    G[cq[2]][cq[3]] = 0
                    G[cq[3]][cq[2]] = 0
        t4 = time.time()
    print(f"{t2-t1} : {t3-t2} : {t4-t3}")
    return G

In [ ]:
G = no4CliqueG_analyze(n, p)

We gotta worry about one combinatorial definition:\
**Def** A $K_4$ (4-clique) between vertices $w,x,y,z \in V$ in a graph G is defined as the edges $(w,x),(w,y),(w,z),(x,y),(x,z),(y,z) \in E$

So... how about we jsut add edges into our graph and check to make sure we dont CREATE a 4-clique? Construction below

In [ ]:
# I believe we want our graph to be connected. So, let us start with a list [n] of our vertices,
# and using a while loop to just keep adding to an edge set, imma just do it haha
def makeGraphFast(n, p):
    # I lowkey think we just need the edge set because we know all n vertices exist lolz
    E = set()
    connect = list(range(n))
    connected = [connect[0]]
    random.shuffle(connect)
    for i in range(1, n):
        u = connect[i]
        v = random.choice(connected)
        if u < v:
            E.add((u, v))
        else:
            E.add((v, u))
        connected.append(u)

    edgeMax = math.comb(n, 2)
    edgeMax = int(p*(edgeMax))
    for _ in range(edgeMax - n):
        v = int(random.random() * n)
        u = int(random.random() * n)
        if not u == v:
            if u < v:
                E.add((u,v))
            else:
                E.add((v,u))
    return E

In [ ]:
# I believe we want our graph to be connected. So, let us start with a list [n] of our vertices,
# and using a while loop to just keep adding to an edge set, imma just do it haha

#This works up to input n=1000 reasonably; cannot handle 10000 :(
def makeGraphFast2(n, p):
    # I lowkey think we just need the edge set because we know all n vertices exist lolz
    E = []
    for i in range(n):
        E.append(set())
    connect = list(range(n))
    random.shuffle(connect)
    connected = [connect[0]]
    for i in range(1, n):
        u = connect[i]
        v = random.choice(connected)
        E[u].add(v)
        E[v].add(u)
        connected.append(u)

    edgeMax = math.comb(n, 2)
    edgeMax = int(p*(edgeMax))
    for _ in range(edgeMax - n):
        v = int(random.random() * n)
        u = int(random.random() * n)
        if not u == v:
            if len(E[u]) < 2 or len(E[v]) < 2:
                E[u].add(v)
                E[v].add(u)
        # if not u == v:
        #     neighbors = E[u] & E[v]
        #     flag = True
        #     for i, j in list(combinations(neighbors, 2)):
        #         if j in E[i]:
        #             flag = False
        #     if flag:
        #         E[u].add(v)
        #         E[v].add(u)
    return E

Note the following theorem:
For a randomly instantiated graph G=(V,E) where V=[n], |E| = k, the probability that a randomly added edge creates a 4-clique is:
$\frac{nPr(k,5)}{nPr(nCr(n,2),5)}\times nCr(n,4)$

In [ ]:
# Uses my random formula to help
def makeGraphFast2(n):
    # I lowkey think we just need the edge set because we know all n vertices exist lolz
    E = []
    for i in range(n):
        E.append(set())
    connect = list(range(n))
    random.shuffle(connect)
    connected = [connect[0]]
    for i in range(1, n):
        u = connect[i]
        v = random.choice(connected)
        E[u].add(v)
        E[v].add(u)
        connected.append(u)

    edgeMax = math.comb(n, 2)
    edgeMax = int(p*(edgeMax))
    for _ in range(edgeMax - n):
        v = int(random.random() * n)
        u = int(random.random() * n)
        if not u == v:
            if len(E[u]) < 2 or len(E[v]) < 2:
                E[u].add(v)
                E[v].add(u)
    return E